In [2]:
#Dependencies
import os
import io
import numpy as np
import glob

import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

import keras 
from keras.preprocessing import image
from keras.models import Model
from keras.utils import to_categorical
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import (VGG19, preprocess_input, decode_predictions)

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
# Load VGG19 model, exclude top layer 
baseModel = VGG19(include_top=False, weights='imagenet')

In [4]:
# Add the last layer 
num_classes = 8
x = baseModel.output
x = GlobalAveragePooling2D()(x)
# add a Fully Connected Dense Layer of size 1024
x = Dense(1024, activation='relu')(x)
# add softmax on the output 
predictions = Dense(num_classes, activation='softmax')(x)
# new keras model with last layer 
model = Model(input=baseModel.input, output=predictions)

C:\Users\rupal\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [5]:
# Freeze all layers from basemodel to exclude from training
for layer in baseModel.layers:
    layer.trainable = False
# compile the new model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',  metrics=['accuracy'])  

In [6]:
train_batch = 'images_split/train'
valid_batch = 'images_split/valid'
test_batch = 'images_split/test'

In [13]:
#grabs images from the folders
train_batches = ImageDataGenerator().flow_from_directory(directory=train_batch,
                            target_size=(224,224), 
                            classes=['animals','baches','birds','electronics','fashion','nature','landmarks','people'], 
                            batch_size=10)
valid_batches = ImageDataGenerator().flow_from_directory(directory=valid_batch,
                            target_size=(224,224),  
                            classes=['animals','beaches','birds','electronics','fashion','nature','landmarks','people'],                             
                            batch_size=8)    
test_batches = ImageDataGenerator().flow_from_directory(directory=test_batch,
                            target_size=(224,224),  
                            classes=['animals','beaches','birds','electronics','fashion','nature','landmarks','people'],                           
                            batch_size=10)                                                   

Found 1409 images belonging to 8 classes.
Found 32 images belonging to 8 classes.
Found 374 images belonging to 8 classes.


In [12]:
#fits the model on data generator batch by batch
model.fit_generator(train_batches, steps_per_epoch=486, validation_data=valid_batches, validation_steps=8, epochs=3, verbose=2)
#model.fit_generator(train_batches, steps_per_epoch=243, validation_data=valid_batches, validation_steps=8, epochs=6, verbose=2)

In [ ]:
#model.save('VGG19_trained_model.h5')